In [84]:
#!/usr/bin/env python
# coding: utf-8

# load a bunch of stuff
from __future__ import division
# load
import numpy as np
import scipy
import pylab
import matplotlib
import matplotlib.pyplot  as plt
import matplotlib.gridspec as gridspec
from matplotlib.pyplot import cm
from matplotlib.ticker import NullFormatter, MaxNLocator, LogLocator
import matplotlib.patches as mpatches
import matplotlib.lines as mlines
plt.switch_backend('agg')  # needed for saving figures
import csv
from pydas.dassl import DASSL
import os
import rmgpy
import rmg
import re
import operator
import pandas as pd
import pylab
from cycler import cycler
import seaborn as sns
import os
import multiprocessing
import itertools

In [2]:
# set up the LSR grid
carbon_range = (-8.0, -2.0)
oxygen_range = (-6.5, -1.5)
grid_size = 9
mesh  = np.mgrid[carbon_range[0]:carbon_range[1]:grid_size*1j,
                 oxygen_range[0]:oxygen_range[1]:grid_size*1j]

with sns.axes_style("whitegrid"):
    plt.axis('square')
    plt.xlim(carbon_range)
    plt.ylim(oxygen_range)
    plt.yticks(np.arange(-6.5,-1,0.5))
plt.show()

# just to double-check
experiments = mesh.reshape((2,-1)).T

with sns.axes_style("whitegrid"):
    plt.axis('square')
    plt.xlim(carbon_range)
    plt.ylim(oxygen_range)
    plt.yticks(np.arange(-6.5,-1.,0.5))
    plt.plot(*experiments.T, marker='o', linestyle='none')
plt.clf()
extent = carbon_range + oxygen_range

# Because the center of a corner pixel is in fact the corner of the grid
# Becaus we want to stretch the image a little
c_step = mesh[0,1,0]-mesh[0,0,0]
o_step = mesh[1,0,1]-mesh[1,0,0]
carbon_range2 = (carbon_range[0]-c_step/2, carbon_range[1]+c_step/2)
oxygen_range2 = (oxygen_range[0]-c_step/2, oxygen_range[1]+c_step/2)
extent2 = carbon_range2 + oxygen_range2

/Users/emilymazeau/anaconda3/envs/rmg/lib/python3.7/site-packages/ipykernel_launcher.py:13: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  del sys.path[0]


In [3]:
# For close packed surfaces from
# Abild-Pedersen, F.; Greeley, J.; Studt, F.; Rossmeisl, J.; Munter, T. R.;
# Moses, P. G.; Skúlason, E.; Bligaard, T.; Norskov, J. K.
# Scaling Properties of Adsorption Energies for Hydrogen-Containing Molecules on
# Transition-Metal Surfaces. Phys. Rev. Lett. 2007, 99 (1), 016105
# DOI: 10.1103/PhysRevLett.99.016105.
abildpedersen_energies = { # Carbon, then Oxygen
    'Pt':(-6.363636363636363,-3.481481481481482),
    'Rh':(-6.5681818181818175,-4.609771721406942),
    'Ir':(-6.613636363636363,-5.94916142557652),
    'Au':(-3.7499999999999973,-2.302236198462614),
    'Pd':(-6, -3.517877940833916),
    'Cu':(-4.159090909090907,-3.85272536687631),
    'Ag':(-2.9545454545454533,-2.9282552993244817),
    'Ni':(-6.045454545454545,-4.711681807593758),
    'Ru':(-6.397727272727272,-5.104763568600047),
}

# "A Framework for Scalable Adsorbate-adsorbate Interaction Models"
# Max J Hoffmann, Andrew J Medford, and Thomas Bligaard
# From 2016 Hoffman et al.
# https://doi.org/10.1021/acs.jpcc.6b03375
hoffman_energies = { # Carbon, then Oxygen
    'Pt':(-6.750,-3.586),
    'Rh':(-6.78,-5.02),
    'Ir':(-6.65,-4.73),
    'Pd':(-6.58,-4.38),
    'Cu':(-4.28,-4.51),
    'Ag':(-2.91,-3.55),
}

katrin_energies = {# Carbon, then Oxygen
    'Pt':(-7.02516,-3.81153),
    'Rh':(-7.33484,-4.71419),
    'Ir':(-7.25234,-4.35236),
    'Au':(-4.5465,-2.71822),
    'Pd':(-7.16786,-4.13577),
    'Cu':(-4.96034,-4.20764),
    'Ag':(-3.50609,-3.11159),
    'Ni':(-6.79794,-4.98902),
    'Ru':(-7.5979,-5.4492),
}

In [93]:
def plot_coords(energies, label, show_text=True):
    """
    Plots binding energy coordinates given in dict format.
    
    show_text is `True` to display text label on plot.
    """

    markers = {
        'Abild-Pedersen':'o',
        'Hoffman':'s',
        'RMG':'X',
    }
    
    colors = {
        'Pt':'r',
        'Rh':'darkorange',
        'Ir':'limegreen',
        'Au':'darkgreen',
        'Pd':'dodgerblue',
        'Cu':'blue',
        'Ag':'darkviolet',
        'Ni':'magenta',
        'Ru':'deeppink',
    }
    
    for metal, coords in energies.items():
        plt.plot(coords[0], coords[1], marker=markers[label], color=colors[metal], label=label)
        if show_text is True:
            plt.text(coords[0], coords[1]-0.15, metal, color=colors[metal])

plot_coords(abildpedersen_energies,'Abild-Pedersen',)
plot_coords(hoffman_energies,'Hoffman',)
plot_coords(katrin_energies,'RMG',)

plt.xlim(carbon_range)
plt.ylim(oxygen_range)
plt.yticks(np.arange(-6.5,-1,1))
plt.xlabel('$\Delta E^C$ (eV)',fontsize=18)
plt.ylabel('$\Delta E^O$ (eV)',fontsize=18)

o_marker = mlines.Line2D([], [], color='k', marker='o', label='Abild-Pedersen', linestyle="None")
s_marker = mlines.Line2D([], [], color='k', marker='s', label='Hoffman', linestyle="None")
x_marker = mlines.Line2D([], [], color='k', marker='X', label='RMG', linestyle="None")

plt.legend(handles=[o_marker,s_marker,x_marker],loc='lower right')
plt.xticks(fontsize=16)
plt.yticks(fontsize=16)
plt.rcParams["figure.figsize"]=(6,6)
plt.tight_layout()
plt.savefig('binding_energies_marker.pdf', bbox_inches='tight')
plt.clf()

In [94]:
def plot_coords(energies, label, show_text=True):
    """
    Plots binding energy coordinates given in dict format.
    
    show_text is `True` to display text label on plot.
    """

    colors = {
        'Abild-Pedersen':'g',
        'Hoffman':'b',
        'RMG':'r',
    }
    
    markers = {
        'Pt':'o',
        'Rh':'v',
        'Ir':'^',
        'Au':'<',
        'Pd':'>',
        'Cu':'s',
        'Ag':'X',
        'Ni':'D',
        'Ru':'P',
    }
    
    for metal, coords in energies.items():
        plt.plot(coords[0], coords[1], marker=markers[metal], color=colors[label], label=label)
        if show_text is True:
            plt.text(coords[0], coords[1]-0.15, metal, color=colors[label])

plot_coords(abildpedersen_energies,'Abild-Pedersen',)
plot_coords(hoffman_energies,'Hoffman',)
plot_coords(katrin_energies,'RMG',)

plt.xlim(carbon_range)
plt.ylim(oxygen_range)
plt.yticks(np.arange(-6.5,-1,1))
plt.xlabel('$\Delta E^schoC$ (eV)',fontsize=18)
plt.ylabel('$\Delta E^O$ (eV)',fontsize=18)

green_patch = mpatches.Patch(color='g',label='Abild-Pedersen')
blue_patch = mpatches.Patch(color='b',label='Hoffman')
red_patch = mpatches.Patch(color='r',label='RMG')
plt.legend(handles=[green_patch,blue_patch,red_patch],loc='lower right',)
plt.xticks(fontsize=16)
plt.yticks(fontsize=16)
plt.rcParams["figure.figsize"]=(6,6)
plt.tight_layout()
plt.savefig('binding_energies_color.pdf', bbox_inches='tight')
plt.clf()